In [1]:
import AutoSummary as ausu

In [2]:
content = 'issue1.txt'  
with open(content, 'r', encoding='utf8') as f:  #讀取原始文章
    text = f.read()

stops = []
with open('dictionary/stopWord_summar.txt','r', encoding='utf8') as f:  #停用詞庫
    for line in f.readlines():
        stops.append(line.strip())

sentences,indexs = ausu.split_sentence(text)  #按標點分割句子split_sentence預設值為[!?。！？]
tfidf = ausu.get_tfidf_matrix(sentences,stops)  #移除停用詞並轉換為矩陣(tfidf) get_tfidf_matrix，stops為停用詞庫
word_weight = ausu.get_sentence_with_words_weight(tfidf)  #計算句子關鍵詞權重get_sentence_with_words_weight
posi_weight = ausu.get_sentence_with_position_weight(sentences)  #計算位置權重get_sentence_with_position_weight
scores = ausu.get_similarity_weight(tfidf)  #計算相似度權重get_similarity_weight
sort_weight = ausu.ranking_base_on_weigth(word_weight, posi_weight, scores, feature_weight = [1,1,1])  #按句子權重排序ranking_base_on_weigth
summar = ausu.get_summarization(indexs,sort_weight,topK_ratio = 0.1)  #取得摘要get_summarization
print('原文:\n', text)
print('==========================================================')
print('摘要:\n',summar)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.753 seconds.
Prefix dict has been built succesfully.


原文:
 ﻿台南出生的黃仁勳，25 年前在美國創辦的 NVIDIA，近年來屢創高峰，股價成長超過 700％、營收數字成長 94％。而黃仁勳在本刊英文版的全球執行長 100 強的排行也節節高升，從三年前的第 29 強，到今年已衝上第 2 強。

我是一個守護者，探索無垠的宇宙，保護這個星球的安全，這個所有生物稱為『家』的地方。」

台灣時間雙十國慶的傍晚， 在德國慕尼黑，NVIDIA（輝達）執行長黃仁勳（JensenHuang）， 正準備上台發表 2018 第三場的全球 GPU 技術大會（GPU TechnologyConference，簡稱 GTC）。

在他上台之前，一段搭配人工語音旁白的影片，以感性的訴求，揭開了大會的序幕。在這段影片中，醫療行為、工業製造、車輛駕駛和音樂創作，都可以由機器取代人類。最後告訴大家，這就是人工智慧，藉由 NVIDIA 創造的深度學習和卓越心智，可以達成的遠大目標。

在音樂和掌聲中出場的 Jensen Huang，仍穿著他的招牌黑色皮衣，從容地步上舞台。

在這場演講中，黃仁勳展示了 NVIDIA 在無人車領域的研發成果。由 NVIDIA 團隊打造的無人車，已能在交通繁忙的矽谷高速公路上行駛八十公里，而且中途不需任何真人駕駛介入操控。

黃仁勳也談到醫療的應用，尤其是在許多生物影像的判讀上，人工智慧可以讓醫師的工作變得更省力卻更精準；最後，他強調虛擬實境（VR）已跳脫遊戲的範疇，營建業可用來創造建築實景，車廠也可以拿來設計超跑。

他一個人撐滿兩小時的演講，會後還獨自面對來自媒體、產業分析師和各公司代表的提問。

從 2009 年開始舉辦的 GTC 全球巡迴，迄今已成為觀察 NVIDIA 發展情況的最重要機會，而黃仁勳正是其中最大的亮點。在成為全球人工智慧的代表廠商後，黃仁勳的動態，更是備受關注。

黃仁勳生於台灣台南，九歲赴美，三十歲創立 NVIDIA，今年 55 歲，操著流利英語，每次回台灣面對媒體時，三不五時夾雜幾句國、台語，強調自己是「台灣囝仔」。

這位台灣囝仔，近年可是大紅大紫，能見度不斷提升。在本期《哈佛商業評論》的「全球執行長 100 強」排行中，今年他更是高居世界第二，僅次於西班牙 ZARA 母公司 Inditex 公司執行長帕布羅．伊斯拉（PabloIsla）。

自本刊全球執行長 100 強調查六年以來，黃仁勳

In [3]:
import requests
from bs4 import BeautifulSoup as soup

stops = []
with open('dictionary/stopWord_summar.txt','r', encoding='utf8') as f:  #停用詞庫
    for line in f.readlines():
        stops.append(line.strip())

urls = []
url = 'https://udn.com/news/breaknews/1'  #聯合報新聞
html = requests.get(url)
sp = soup(html.text, 'html.parser')
data1 = sp.select('#breaknews_body dl dt h2 a')
for d in data1:  #取得新聞連結
    urls.append('https://udn.com' + d.get('href'))

i = 1
for url in urls:  #逐一取得新聞
    html = requests.get(url)
    sp = soup(html.text, 'html.parser')
    data1 = sp.select('#story_body_content p')  #新聞內容
    print('處理第 {} 則新聞'.format(i))
    text = ''
    for d in data1:
        if d.text.find('延伸閱讀') != -1:  #遇到延伸閱讀就結束此則新聞
            break
        if d.text != '':  #有新聞內容
            text += d.text
    sentences,indexs = ausu.split_sentence(text)  #按標點分割句子
    tfidf = ausu.get_tfidf_matrix(sentences,stops)  #移除停用詞並轉換為矩陣
    word_weight = ausu.get_sentence_with_words_weight(tfidf)  #計算句子關鍵詞權重
    posi_weight = ausu.get_sentence_with_position_weight(sentences)  #計算位置權重
    scores = ausu.get_similarity_weight(tfidf)  #計算相似度權重
    sort_weight = ausu.ranking_base_on_weigth(word_weight, posi_weight, scores, feature_weight = [1,1,1])
    summar = ausu.get_summarization(indexs,sort_weight,topK_ratio = 0.3)  #取得摘要
    print(summar)
    print('==========================================================')
    i += 1
    

處理第 1 則新聞
2020年馬上到來，臉書破10萬粉絲追蹤的「報時光」粉專，精選2019年最獲粉絲好評的五大精選老照片，帶民眾一起搭乘時光機，從回味美好舊時光裡，找到更多2020年向前邁進的動力。 分享   facebook     1988年刊登在民生報校園版的「中學生 嗜打電視遊樂器｣新聞，探討中學生流行「打任天堂」，交換「祕笈」和卡帶，到店裡打行情價是1分鐘1塊錢，1小時50塊。有四年級粉絲拿出當成傳家寶的「紅白機｣說：「我現在還在玩｣；更多引發共鳴的是大家分享放學玩電動被教官、爸媽抓的回憶：「被老師抓到打了20大板...」、「國中時期偷跑去玩，破關很高興！ 分享   facebook     「嗶啵～嗶啵～」1990年由潘迎紫主演的《浴火鳳凰》有個守護神獸「嘟嘟」，當年風靡家家戶戶，所有小孩口頭禪都成了「嗶啵～嗶啵～」連買油還會送嗶啵玩具，你知道它其實是日本製作設計的嗎？」3.年度話題王【大安森林公園的前身】7號公園(即現今大安森林公園)預定地內軍方眷村居民強烈抗爭。 分享   facebook     這張1990年8月拍攝的新聞照片，描述7號公園（即現今大安森林公園）預定地內，眷村居民強烈抗爭阻撓拆遷，也是首度讓人一窺大安森林公園前身的大場景照片。5.年度最古早味【塑膠袋裝飲料，勾起你的懷舊回憶】在牛肉麵街裡，有三家冰果攤，這裡的果汁和牛肉麵有著相同的特點－「便宜又大袋」。
處理第 2 則新聞
樂天桃猿隊今天公布新球季教練團，近年幾乎固定來台擔任中華隊、中職球隊教練的克魯茲（Tommy Curz）再度回鍋，明年將以客座打擊教練的身分入隊，預定從春訓待到上半季前半段，他早就說過：「台灣是我的第二個家。
處理第 3 則新聞



E:\Machin_learning_Deep_learning\practice5\AutoSummary.py:44: RuntimeWarning: invalid value encountered in double_scalars
  sentence_with_words_weight[key] = (x-min_weight)/(max_weight-min_weight)
E:\Machin_learning_Deep_learning\practice5\AutoSummary.py:68: RuntimeWarning: invalid value encountered in double_scalars
  sentence_score[key] = (x-min_score)/(max_score-min_score)


處理第 4 則新聞
日本網球明星錦織圭決定退出首屆世界職業網球協會盃（ATP Cup）。 歐新社 分享   facebook     日本網球明星錦織圭因為從美國公開賽（US Open）就開始持續困擾他至今的手肘傷勢，在今天決定退出首屆世界職業網球協會盃（ATP Cup）。世界排名曾排在第4，目前滑落到第13名的錦織圭（Kei Nishikori）表示，對於不能代表自己的國家，參加2020年1月3日開始，在澳洲伯斯（Perth）進行的團體賽事，他感到很失望。
處理第 5 則新聞
受到雨勢影響，新北富邦國際城市U18棒球邀請賽四強戰提前於七局裁定，由新竹市隊以5：2擊敗花蓮縣隊。記者蘇志畬／攝影 分享   facebook     新北富邦國際城市U18棒球邀請賽四強戰受到雨勢影響，比賽提前於七局結束，裁定由新竹市隊以5：2擊敗花蓮縣隊，明天冠軍賽將遭遇以穀保家商組成的新北藍隊。
處理第 6 則新聞
美國紐約北部蒙西市（Monsey）在28日晚間發生針對猶太人的攻擊事件，嫌犯湯瑪斯（Grafton Thomas）闖入猶太教拉比羅騰堡（Chaim Rottenberg）的住家，當時屋內正在舉辦光明節儀式，湯瑪斯持刀隨機砍人，共造成5人受傷，其中2人傷勢嚴重。事件發生在28日晚間11點45分左右，當時在羅騰堡的家中大約聚集超過100人，正在進行點蠟燭的儀式，凶手湯馬斯闖入屋內，拿出長刀當凶器，開始隨機砍人，造成5人受傷，2人傷勢嚴重，其中一名受害者是拉比的兒子。美國紐約北部蒙西市（Monsey）在28日晚間發生針對猶太人的攻擊事件，嫌犯闖入猶太教拉比羅騰堡（Chaim Rottenberg）的住家，當時屋內正在舉辦光明節儀式，湯瑪斯持刀隨機砍人，共造成5人受傷，其中2人傷勢嚴重。
處理第 7 則新聞
親民黨總統候選人宋楚瑜和鴻海創辦人郭台銘直播吃早餐，還點了菜包吃，被聯想是不是在暗諷總統蔡英文。 圖／翻攝郭台銘臉書 分享   facebook     鴻海創辦人郭台銘今天與親民黨總統候選人宋楚瑜一同早餐，並說出「棄韓保宋」。郭台銘幕僚蔡沁瑜表示，郭台銘只是在複誦宋楚瑜講的話，由於事涉郭台銘個人的投票意向，不便代為發言。郭台銘、宋楚瑜今天上午相約吃早餐並直播，有網友留言「棄韓保宋」，宋楚瑜說，「太好了，棄韓保宋，感謝你」，而郭台銘也在旁複誦「棄韓保宋」，引發各界討論，「棄韓

處理第 27 則新聞
記者李京昇／攝影 分享   facebook     韓國瑜妻舅李明哲及候選人謝淑亞在造勢場合，公開指民進黨政府將通過「吸毒無罪」，四處造謠吃毒無罪，涉散播不實謠言，誤導青少年認為吃毒無罪，嚴重影響校園和社會，也詆譭民進黨名譽，民進黨雲林縣黨部今天到雲林地檢署按鈴控告兩人違反社會秩序法與誹謗罪。林廉貴說，新法追訴門檻，明明比舊法更嚴格，但韓國瑜的妻舅也是雲林縣議員李明哲卻在造勢場合上，公開說「民進黨將修法讓吃毒品無罪，大家應予唾棄...」等言論，不僅造成社會動盪不安，也嚴重影響青少年心理，誤導以為「吃毒無罪」，也讓法務部多年的反毒破攻，法務部也已鄭重澄清，絕無「吃毒無罪」之事。林廉貴並說，在造勢場合之外，雲林縣副縣長也是立委候選人謝淑亞隨後又開記者會，再度提到「5公克以下毒品可以在校園賣」等荒謬言論，意圖鼓勵校園販毒，李明哲和謝淑亞的公開言論，非但嚴重影響社會，涉及違反社會秩序維護法，散佈謠言足以影響公共之安寧者，且以不實言論詆譭民進黨，已涉及誹，因而今天控告兩人誹謗和違反社維法。
處理第 28 則新聞
屏東縣第二選區立委公辦政見會下午錄影，由於候選人蘇震清與周佳琪到錄影前一刻仍在捉對廝殺，使得政見會更受注目，不過周佳琪並未到場錄影，首先發表政見的蘇震清在政見會前再度回應對手的指控，同時重申如果自己有投資砂石廠做違法事情，「退出政壇」，並要議長周典論和女兒周佳琪一起出手，他隨時奉陪。屏東縣第二選區總計有9人登記參選，首先發表政見的蘇震清除了回應對手指控外，並主張未來會推動高捷南延屏東，全面提升屏縣自來水普及率、加強公共運輸，妥善連結屏南醫療與社區關懷據點。屏東縣第二選區立委公辦政見會今天下午錄影，除國民黨周佳琪另有行程未到場外，其餘8位候選人均到場。
處理第 29 則新聞
善心人士李福田今天捐贈兩台社福溫馨交通車給雲林縣身心障礙團體，雲林縣長張麗善代表受贈並回贈感謝狀，以感謝李福田善心義舉。 分享   facebook     善心人士李福田今天捐贈兩台社福溫馨交通車給雲林縣身心障礙團體，雲林縣長張麗善代表受贈並回贈感謝狀，以感謝李福田善心義舉。
處理第 30 則新聞
澳洲戰略政策研究所（Australian StrategicPolicy Institute）防衛策略及國家安全研究主任休布瑞吉（Michael Shoebridge）。 中央社

處理第 49 則新聞
時序入冬，低溫加雨水催化，南投縣國姓鄉九份二山梅花相繼盛放，山頭換上雪白新妝，當地3大梅區平均花況近7成，元旦進入最佳遊賞期，公所將從當天上午8時起進行交通疏導，限制由龍南路進、長石巷出，全面管制至7日下午4時。記者賴香珊／攝影 分享   facebook     南投縣國姓鄉九份二山梅花盛放，山頭換上雪白新妝，當地梅區平均花況近7成，元旦起進入最佳遊賞期。記者賴香珊／攝影 分享   facebook     南投縣國姓鄉九份二山梅花盛放，山頭換上雪白新妝，當地梅區平均花況近7成，元旦起進入最佳遊賞期。
處理第 50 則新聞
因應108學年度課綱起跑，桃園市私立大華中學日前舉辦歲未年終「數位起飛、素養啟程」成果發表會，邀請數位教育推廣專家潘乾芳以「數位時代教養攻略」為題演講，為活動揭開序幕，接著以各領域國中部課程為主，開放教室，邀請校內外專家、教師、家長一同參與課程。張佩瑜表示，每年12月是大華一系列的教學活動成果觀摩，也成為大華中學的傳統和傳承，每年歲末，校園中總是充滿歡慶佳節、感恩、分享、祝福的氛圍，從高中部的英文戲劇、國中部的班際合唱比賽等藝文饗宴，加上各領域豐富多元的教學活動設計，引導學生進行歲末的反思回饋。此外，各科都在各自的專科教室展示學生學習成果，也藉此檢視學生們是否達成今年學校培養科技數位素養─「看見改變，擁抱未來」的教學目標。成果發表會另一個重頭戲是學生展演，除了有大華中學的弦樂團及桃園市合唱比賽的常勝軍─女聲組合唱團的演奏演唱，焦點之一是高中部必修課程「英文戲劇」成果展演。大華中學從國小、國中到高中有一貫的教學系統，強調品格、英語、科學、人文、探索等多元活化的教學，目的在培養學生運用科技，自主學習，讓大華學生在學習上應用創客精神，豐富學習歷程，真正實踐「以傳統邁向國際，以科技迎向未來」的目標。大華中學今年英文戲劇成果展演出英國作家狄更斯著名作品「孤雛淚(Oliver Twist)」，學生全程英語演出。
處理第 51 則新聞
民進黨新竹市立委候選人鄭宏輝今天下午針對國民黨不斷對民進黨進行抹紅、抹黑攻勢，及時代力量抹黑與不實指控，至新竹地檢署提告國民黨立委候選人鄭正鈐、時代力量立委候選人高鈺婷，違反選罷法及加重誹謗。鄭宏輝表示，國民黨抨擊他曾赴陸投資公司是烏賊戰術，這樣的抹黑等於是將所有正常經商的台商扣上大紅帽，鄭宏輝說，民進黨